# Trabajo Practico 2 - Grupo 04 - Backbenchers

**Integrantes:**


* Catalina Basso
* Cristobal Alvarez
* Lucas Ruiz
* Dalmiro Vilaplana  

## Imports

In [ ]:
from google.colab import drive
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.datasets import make_classification
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Descargas de nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

## Funciones

In [ ]:
# Eliminar puntuación y convertir a minúsculas
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub("\d+", "", text)
    text = text.strip()
    return text

# Eliminar palabras vacías
stop_words = set(stopwords.words('spanish'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

def limpiar_texto_leman(texto):
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = texto.lower()
    texto = re.sub(r'\d+', '', texto)
    stop_words = set(stopwords.words('spanish'))
    palabras = texto.split()
    palabras = [palabra for palabra in palabras if palabra not in stop_words]
    lemmatizer = WordNetLemmatizer()
    palabras = [lemmatizer.lemmatize(palabra) for palabra in palabras]
    return ' '.join(palabras)


stemmer = SnowballStemmer('spanish')

def stem_text(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

# Lista de palabras de parada en español
stop_words_spanish = [
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para',
    'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o',
    'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también',
    'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos',
    'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto',
    'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto',
    'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar',
    'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus',
    'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías',
    'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro',
    'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras',
    'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'esté',
    'estés', 'estemos', 'estéis', 'estén', 'estaré', 'estarás', 'estará', 'estaremos',
    'estaréis', 'estarán', 'estaría', 'estarías', 'estaríamos', 'estaríais', 'estarían',
    'estaba', 'estabas', 'estábamos', 'estabais', 'estaban', 'estuve', 'estuviste',
    'estuvo', 'estuvimos', 'estuvisteis', 'estuvieron', 'estuviera', 'estuvieras',
    'estuviéramos', 'estuvierais', 'estuvieran', 'estuviese', 'estuvieses', 'estuviésemos',
    'estuvieseis', 'estuviesen', 'estando', 'estado', 'estada', 'estados', 'estadas',
    'estad'
]

def preprocess_data(data):
    label_encoder = LabelEncoder()
    data['sentimiento'] = label_encoder.fit_transform(data['sentimiento'])

    tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words_spanish)
    X = tfidf_vectorizer.fit_transform(data['review_es'])
    y = data['sentimiento']

    return X, y, tfidf_vectorizer, label_encoder

## Lectura CSV


Primero cargamos y vemos nuestro dataset

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_train.csv')
train.head()

ID                                          review_es sentimiento
0   0  Uno de los otros críticos ha mencionado que de...    positivo
1   1  Una pequeña pequeña producción.La técnica de f...    positivo
2   2  Pensé que esta era una manera maravillosa de p...    positivo
3   3  Básicamente, hay una familia donde un niño peq...    negativo
4   4  El "amor en el tiempo" de Petter Mattei es una...    positivo

In [ ]:
train.shape

(50000, 3)

In [ ]:
test_kaggle = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_test.csv')
test_kaggle.head()

ID                                          review_es
0  60000  La mayor virtud de esta película es su existen...
1  60001  No soy un experto cinéfilo, pero pocas veces m...
2  60002  Si no eres un incondicional del humor estilo T...
3  60003  No sé qué está pasando, si la gente se deja ll...
4  60004  Pero cuando amanece,y me quedo solo,siento en ...

In [ ]:
test_kaggle.shape

(8599, 2)

# **Bagging**

## Ensamble 1: Bagging + Random Forest

vamos a tomar el modelo de random forest que mejores resultados obtuvo en las submissions anteriores, y lo utilizaremos para nuestro modelo de ensamble.

### Algoritmo

#### Paso 1: Preprocesamiento de datos


In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords)

#### Paso 2: Aplico Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

vamos a mapear nuestras variables negativo y positivo a 0 y 1 para poder trabajar con el modelo

In [ ]:
label_map = {'negativo': 0, 'positivo': 1}
y_map = [label_map[label] for label in y]

#### Paso 3: Division

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

#### Paso 4: Entreno mi modelo

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=5)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=5)

In [ ]:
bagging_classifier = BaggingClassifier(clf, n_estimators=3, random_state=5)

# Train the bagging classifier
bagging_classifier.fit(X_train, y_train)

BaggingClassifier(estimator=RandomForestClassifier(random_state=5),
                  n_estimators=3, random_state=5)

In [ ]:
y_pred = bagging_classifier.predict(X_test)

#### Paso 5: Evaluo mi modelo

In [ ]:
#vemos las metricas en el conjunto de entrenamiento
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    negativo       0.81      0.84      0.82      4900
    positivo       0.84      0.82      0.83      5100

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



### Prediccion para el modelo de test dado

In [ ]:
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

#vectorizamos para poder utilizar nuestro conjunto de palabras en nuestras predicciones.
vectorizer = CountVectorizer(max_features=5000)
X_nuevos = vectorizer.fit_transform(test_kaggle['review_clean']).toarray()

# Hacer predicciones en los nuevos datos utilizando el modelo entrenado
y_pred = bagging_classifier.predict(X_nuevos)

# Agregar las predicciones al DataFrame de los nuevos datos
test_kaggle['sentimiento'] = y_pred

columnas_mostrar = ['ID', 'sentimiento']
resultado_final_ensamble_bagging = test_kaggle[columnas_mostrar]

print(resultado_final_ensamble_bagging)

         ID sentimiento
0     60000    negativo
1     60001    negativo
2     60002    negativo
3     60003    negativo
4     60004    positivo
...     ...         ...
8594  68594    positivo
8595  68595    negativo
8596  68596    negativo
8597  68597    positivo
8598  68598    negativo

[8599 rows x 2 columns]


In [ ]:
resultado_final_ensamble_bagging.to_csv('ensamble_bagging_basico.csv', index=False)

## Ensamble 2: Bagging + random forest + cross-validation

### Algoritmo

#### Paso 1: Preprocesamiento de datos


In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords)

#### Paso 2: Aplico Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

vamos a mapear nuestras variables negativo y positivo a 0 y 1 para poder trabajar con el modelo

In [ ]:
label_map = {'negativo': 0, 'positivo': 1}
y_map = [label_map[label] for label in y]

#### Paso 3: Division

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

#### Paso 4: Creo mi modelo

In [ ]:
#Cantidad de combinaciones que quiero probar
n=5

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],
               'max_depth':list(range(1,20)), #ponemos un rango de 20 porque tenemos muchas columnas que evaluar
               'ccp_alpha':np.linspace(0,0.05,n), #ccp para evitar overfitting
               'n_estimators': list(range(1,20))
               }

#Cantidad de splits para el Cross Validation
folds=5

#Kfold estratificado
kfoldcv = StratifiedKFold(n_splits=folds)

# creamos un random forest classifier
rf = RandomForestClassifier()

#Random Search Cross Validation
randomForest = RandomizedSearchCV(estimator=rf,
                              param_distributions = params_grid,
                              scoring='f1_macro',
                               cv=kfoldcv,
                              n_iter=n,
                              random_state = 20
                              )

randomForest.fit(X_train,y_train);

#### Paso 5: Entreno mi modelo

In [ ]:
#creando el randomForest con los mejores parametros
random_forest_mejor = RandomForestClassifier().set_params(**randomForest.best_params_)
random_forest_mejor.fit(X_train,y_train)

RandomForestClassifier(max_depth=15, n_estimators=6)

In [ ]:
# Vemos los mejores hiperparametros
print('Mejores Hiperparametros:',  randomForest.best_params_)

Mejores Hiperparametros: {'n_estimators': 6, 'max_depth': 15, 'criterion': 'gini', 'ccp_alpha': 0.0}


In [ ]:
bagging_classifier = BaggingClassifier(random_forest_mejor, n_estimators=3, random_state=5)

# Train the bagging classifier
bagging_classifier.fit(X_train, y_train)

BaggingClassifier(estimator=RandomForestClassifier(max_depth=15,
                                                   n_estimators=6),
                  n_estimators=3, random_state=5)

In [ ]:
y_pred = bagging_classifier.predict(X_test)

#### Paso 6: Evaluo mi modelo

In [ ]:
#vemos las metricas en el conjunto de entrenamiento
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    negativo       0.81      0.70      0.75      4900
    positivo       0.74      0.84      0.79      5100

    accuracy                           0.77     10000
   macro avg       0.77      0.77      0.77     10000
weighted avg       0.77      0.77      0.77     10000



### Prediccion para el modelo de test dado

In [ ]:
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

#vectorizamos para poder utilizar nuestro conjunto de palabras en nuestras predicciones.
vectorizer = CountVectorizer(max_features=5000)
X_nuevos = vectorizer.fit_transform(test_kaggle['review_clean']).toarray()

# Hacer predicciones en los nuevos datos utilizando el modelo entrenado
y_pred = bagging_classifier.predict(X_nuevos)

# Agregar las predicciones al DataFrame de los nuevos datos
test_kaggle['sentimiento'] = y_pred

columnas_mostrar = ['ID', 'sentimiento']
resultado_final_ensamble_bagging_2 = test_kaggle[columnas_mostrar]

print(resultado_final_ensamble_bagging_2)

         ID sentimiento
0     60000    negativo
1     60001    positivo
2     60002    positivo
3     60003    positivo
4     60004    positivo
...     ...         ...
8594  68594    positivo
8595  68595    positivo
8596  68596    positivo
8597  68597    positivo
8598  68598    positivo

[8599 rows x 2 columns]


In [ ]:
resultado_final_ensamble_bagging_2.to_csv('ensamble_bagging_Cross_Validation.csv', index=False)

## Ensamble 3: Bagging + Random Forest + cross-validation + mayor cantidad de estimadores

### Algoritmo

#### Paso 1: Preprocesamiento de datos


In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords)

#### Paso 2: Aplico Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

vamos a mapear nuestras variables negativo y positivo a 0 y 1 para poder trabajar con el modelo

In [ ]:
label_map = {'negativo': 0, 'positivo': 1}
y_map = [label_map[label] for label in y]

#### Paso 3: Division

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

#### Paso 4: Entreno mi modelo

Cambiamos la cantidad de estimadores del modelo de bagging a 10 y observamos los resultados

In [ ]:
bagging_classifier = BaggingClassifier(random_forest_mejor, n_estimators=10, random_state=5)

# Train the bagging classifier
bagging_classifier.fit(X_train, y_train)

BaggingClassifier(estimator=RandomForestClassifier(max_depth=15,
                                                   n_estimators=6),
                  random_state=5)

In [ ]:
y_pred = bagging_classifier.predict(X_test)

#### Paso 5: Evaluo mi modelo

In [ ]:
#vemos las metricas en el conjunto de entrenamiento
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    negativo       0.82      0.75      0.78      4900
    positivo       0.78      0.84      0.81      5100

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



### Prediccion para el modelo de test dado

In [ ]:
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

#vectorizamos para poder utilizar nuestro conjunto de palabras en nuestras predicciones.
vectorizer = CountVectorizer(max_features=5000)
X_nuevos = vectorizer.fit_transform(test_kaggle['review_clean']).toarray()

# Hacer predicciones en los nuevos datos utilizando el modelo entrenado
y_pred = bagging_classifier.predict(X_nuevos)

# Agregar las predicciones al DataFrame de los nuevos datos
test_kaggle['sentimiento'] = y_pred

columnas_mostrar = ['ID', 'sentimiento']
resultado_final_ensamble_bagging_3 = test_kaggle[columnas_mostrar]

print(resultado_final_ensamble_bagging_3)

         ID sentimiento
0     60000    negativo
1     60001    positivo
2     60002    positivo
3     60003    positivo
4     60004    positivo
...     ...         ...
8594  68594    positivo
8595  68595    positivo
8596  68596    positivo
8597  68597    positivo
8598  68598    positivo

[8599 rows x 2 columns]


In [ ]:
resultado_final_ensamble_bagging_3.to_csv('ensamble_bagging_Cross_Validation_Mas_Estimadores.csv', index=False)

# **Voting**

## Ensamble 4: Voting 2

### Algoritmo

#### Paso 1: Preprocesamiento de datos


In [ ]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords)

#### Paso 2: Aplico Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

vamos a mapear nuestras variables negativo y positivo a 0 y 1 para poder trabajar con el modelo

In [ ]:
label_map = {'negativo': 0, 'positivo': 1}
y_map = [label_map[label] for label in y]

#### Paso 3: Division

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train['review_es'], train['sentimiento'], test_size=0.2, random_state=42)

In [ ]:
X_train_clean = X_train.apply(clean_text).apply(remove_stopwords)
X_test_clean = X_test.apply(clean_text).apply(remove_stopwords)

#### Paso 4: Creo las pipelines y el classifier

In [ ]:
# Pipelines para los modelos base
pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords.words('spanish'), max_features=10000, ngram_range=(1, 2))),
    ('clf', BernoulliNB())
])

pipeline_lr = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords.words('spanish'), max_features=10000, ngram_range=(1, 2))),
    ('clf', LogisticRegression(max_iter=200))
])

pipeline_rf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords.words('spanish'), max_features=10000, ngram_range=(1, 2))),
    ('clf', RandomForestClassifier(n_estimators=100))
])

In [ ]:
# Crear el VotingClassifier
voting_clf = VotingClassifier(
    estimators=[
        ('nb', pipeline_nb),
        ('lr', pipeline_lr),
        ('rf', pipeline_rf)
    ],
    voting='soft'  # Usar votación suave (promedio de probabilidades)
)

#### Paso 5: Entreno mi modelo

In [ ]:
# Entrenar el VotingClassifier
voting_clf.fit(X_train_clean, y_train)

VotingClassifier(estimators=[('nb',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_features=10000,
                                                               ngram_range=(1,
                                                                            2),
                                                               stop_words=['de',
                                                                           'la',
                                                                           'que',
                                                                           'el',
                                                                           'en',
                                                                           'y',
                                                                           'a',
                                                                           'los',
                                                                           'del',
                                                                           'se',
                                                                           'las',
                                                                           'por',
                                                                           'un',
                                                                           'para',
                                                                           'con',
                                                                           'no',
                                                                           'una',
                                                                           'su',
                                                                           'al',
                                                                           'lo',
                                                                           'como',
                                                                           'más',
                                                                           'pero',
                                                                           'sus',
                                                                           'le',
                                                                           'ya',
                                                                           'o',
                                                                           'este',
                                                                           'sí',
                                                                           'porque', ...])),
                                              ('clf', BernoulliNB())])),
                             ('lr',
                              Pipeline(steps=[('t...
                                               LogisticRegression(max_iter=200))])),
                             ('rf',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_features=10000,
                                                               ngram_range=(1,
                                                                            2),
                                                               stop_words=['de',
                                                                           'la',
                                                                           'que',
                                                                           'el',
                                                                           'en',
                                                                           'y',
                                                                           'a',
                                                               

#### Paso 6: Evaluo mi modelo

In [ ]:
# Evaluar el modelo
y_pred_voting = voting_clf.predict(X_test_clean)
print("Accuracy del Voting Classifier:", accuracy_score(y_test, y_pred_voting))
print("Informe de clasificación del Voting Classifier:\n", classification_report(y_test, y_pred_voting))

Accuracy del Voting Classifier: 0.8518
Informe de clasificación del Voting Classifier:
               precision    recall  f1-score   support

    negativo       0.86      0.84      0.85      4961
    positivo       0.84      0.86      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar el conjunto de test de Kaggle
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords)

# Hacer predicciones con el VotingClassifier
y_pred_kaggle = voting_clf.predict(test_kaggle['review_clean'])

# Guardar las predicciones para Kaggle
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})
submission.to_csv('Voting_Predictions_2.csv', index=False)

## Ensamble 5: Voting 3

### Algoritmo

#### Paso 1: Division y preprocesamiento de datos

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train['review_es'], train['sentimiento'], test_size=0.2, random_state=0)

In [ ]:
X_train_clean = X_train.apply(clean_text).apply(remove_stopwords)
X_test_clean = X_test.apply(clean_text).apply(remove_stopwords)

#### Paso 2: Creo las pipelines y el classifier

In [ ]:
# Pipelines para los modelos base
pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, ngram_range=(1, 3))),
    ('clf', BernoulliNB())
])

pipeline_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, ngram_range=(1, 3))),
    ('clf', MultinomialNB())
])

pipeline_rf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, ngram_range=(1, 3))),
    ('clf', RandomForestClassifier(n_estimators=100))
])

In [ ]:
# Crear el VotingClassifier
voting_clf = VotingClassifier(
    estimators=[
        ('nb', pipeline_nb),
        ('lr', pipeline_lr),
        ('rf', pipeline_rf)
    ],
    voting='soft'  # Usar votación suave (promedio de probabilidades)
)

#### Paso 3: Entreno mi modelo

In [ ]:
# Entrenar el VotingClassifier
voting_clf.fit(X_train_clean, y_train)

VotingClassifier(estimators=[('nb',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_features=10000,
                                                               ngram_range=(1,
                                                                            3))),
                                              ('clf', BernoulliNB())])),
                             ('lr',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_features=10000,
                                                               ngram_range=(1,
                                                                            3))),
                                              ('clf', MultinomialNB())])),
                             ('rf',
                              Pipeline(steps=[('tfidf',
                                               TfidfVectorizer(max_features=10000,
                                                               ngram_range=(1,
                                                                            3))),
                                              ('clf',
                                               RandomForestClassifier())]))],
                 voting='soft')

#### Paso 4: Evaluo mi modelo

In [ ]:
# Evaluar el modelo
y_pred_voting = voting_clf.predict(X_test_clean)
print("Accuracy del Voting Classifier:", accuracy_score(y_test, y_pred_voting))
print("Informe de clasificación del Voting Classifier:\n", classification_report(y_test, y_pred_voting))

Accuracy del Voting Classifier: 0.8428
Informe de clasificación del Voting Classifier:
               precision    recall  f1-score   support

    negativo       0.85      0.83      0.84      5035
    positivo       0.83      0.86      0.84      4965

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar el conjunto de test de Kaggle
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords)

# Hacer predicciones con el VotingClassifier
y_pred_kaggle = voting_clf.predict(test_kaggle['review_clean'])

# Guardar las predicciones para Kaggle
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})
submission.to_csv('Voting_Predictions_3.csv', index=False)

## Ensamble 7

In [ ]:
predicciones_modelo1 = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulli_v3.csv')
predicciones_modelo2 = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulliv3.5.csv')
predicciones_modelo3 = pd.read_csv('/content/drive/MyDrive/Multinomial_Naive_Bayes_Optimizado.csv')


# Mapeo de etiquetas a valores numéricos
mapeo_etiquetas = {'positivo': 1, 'negativo': 0}

# Convertir las etiquetas a valores numéricos
predicciones_modelo1['sentimiento'] = predicciones_modelo1['sentimiento'].map(mapeo_etiquetas)
predicciones_modelo2['sentimiento'] = predicciones_modelo2['sentimiento'].map(mapeo_etiquetas)
predicciones_modelo3['sentimiento'] = predicciones_modelo3['sentimiento'].map(mapeo_etiquetas)

# Combinar las predicciones de los tres modelos
# Por ejemplo, podrías promediar las probabilidades o tomar la moda de las etiquetas
predicciones_combinadas = (predicciones_modelo1['sentimiento'] + predicciones_modelo2['sentimiento'] + predicciones_modelo3['sentimiento']) / 3

# Redondear las predicciones al entero más cercano, ya que las etiquetas parecen ser valores enteros
predicciones_combinadas = predicciones_combinadas.round().astype(int)

# Mapeo de valores numéricos a etiquetas originales
mapeo_etiquetas_inverso = {1: 'positivo', 0: 'negativo'}

# Mapear los valores numéricos promediados a etiquetas originales
predicciones_promediadas = predicciones_combinadas.map(mapeo_etiquetas_inverso)

# Crear el archivo de envío con las predicciones combinadas
submission_combined = pd.DataFrame({'id': predicciones_modelo1['id'], 'sentimiento': predicciones_promediadas})
submission_combined.to_csv('predicciones_combinadas_3.0.csv', index=False)

## Ensamble 8

In [ ]:
# Cargar las predicciones de los tres modelos
bayes_naive_preds = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulli_v3.csv')['sentimiento']
voting_preds = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulliv3.5.csv')['sentimiento']
neural_net_preds = pd.read_csv('/content/drive/MyDrive/red_neuronal_keras_tensor_flow_basico.csv')['sentimiento']

# Realizar el ensamble de votación
final_predictions = []
for preds in zip(bayes_naive_preds, voting_preds, neural_net_preds):
    final_predictions.append(Counter(preds).most_common(1)[0][0])

# Crear el archivo de envío
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': final_predictions})
submission.to_csv('Ensemble_voting_mejores_predicciones.csv', index=False)

## Ensamble 9

In [ ]:
# Cargar las predicciones de los tres modelos
bayes_naive_preds = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulli_v3.csv')['sentimiento']
voting_preds = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulliv3.5.csv')['sentimiento']
bayes_naive_2 = pd.read_csv('/content/drive/MyDrive/bayes_naive_vector_tomando_varias_palabras_3.csv')['sentimiento']

# Realizar el ensamble de votación
final_predictions = []
for preds in zip(bayes_naive_preds, voting_preds, bayes_naive_2):
    final_predictions.append(Counter(preds).most_common(1)[0][0])

# Crear el archivo de envío
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': final_predictions})
submission.to_csv('Ensemble_voting_mejores_predicciones_2.csv', index=False)

## Ensamble 10

In [ ]:
# Cargar las predicciones de los tres modelos
bayes_naive_preds = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulliv3.5.csv')['sentimiento']
voting_preds = pd.read_csv('/content/drive/MyDrive/Voting_Predictions_3.csv')['sentimiento']
bayes_naive_2 = pd.read_csv('/content/drive/MyDrive/bayes_naive_vector_tomando_varias_palabras_3.csv')['sentimiento']

# Realizar el ensamble de votación
final_predictions = []
for preds in zip(bayes_naive_preds, voting_preds, bayes_naive_2):
    final_predictions.append(Counter(preds).most_common(1)[0][0])

# Crear el archivo de envío
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': final_predictions})
submission.to_csv('Ensemble_voting_promedio_predicciones_3.csv', index=False)

## Ensamble 11: el mejor de cada modelo

In [ ]:
# Cargar las predicciones de los tres modelos
bayes_naive = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulliv3.5.csv')['sentimiento']
random_forest = pd.read_csv('/content/drive/MyDrive/random_forest_smote_modified_2.0.csv')['sentimiento']
xg_boost = pd.read_csv('/content/drive/MyDrive/XGBoost_BagOfWords_trigams_v2.csv')['sentimiento']
red_neuronal = pd.read_csv('/content/drive/MyDrive/red_neuronal_keras_tensor_flow_basico.csv')['sentimiento']
ensamble_voting = pd.read_csv('/content/drive/MyDrive/Voting_Predictions_3.csv')['sentimiento']

# Realizar el ensamble de votación
final_predictions = []
for preds in zip(bayes_naive, random_forest, xg_boost, red_neuronal, ensamble_voting):
    final_predictions.append(Counter(preds).most_common(1)[0][0])

# Crear el archivo de envío
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': final_predictions})
submission.to_csv('voting_promedio_cada_modelo.csv', index=False)

## Ensamble 12: le agrego mas csv's

In [ ]:
# Cargar las predicciones de los tres modelos
bayes_naive = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulliv3.5.csv')['sentimiento']
random_forest = pd.read_csv('/content/drive/MyDrive/random_forest_smote_modified_2.0.csv')['sentimiento']
xg_boost = pd.read_csv('/content/drive/MyDrive/XGBoost_BagOfWords_trigams_v2.csv')['sentimiento']
red_neuronal = pd.read_csv('/content/drive/MyDrive/red_neuronal_keras_tensor_flow_basico.csv')['sentimiento']
ensamble_voting = pd.read_csv('/content/drive/MyDrive/Voting_Predictions_3.csv')['sentimiento']
bayes_naive_2 = pd.read_csv('/content/drive/MyDrive/bayes_naive_vector_tomando_varias_palabras_3.csv')['sentimiento']
bayes_naive_3 = pd.read_csv('/content/drive/MyDrive/Multinomial_Naive_Bayes_Optimizado.csv')['sentimiento']
bayes_naive_4 = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulli_v3.csv')['sentimiento']

# Realizar el ensamble de votación
final_predictions = []
for preds in zip(bayes_naive, random_forest, xg_boost, red_neuronal, ensamble_voting, bayes_naive_2, bayes_naive_3, bayes_naive_4):
    final_predictions.append(Counter(preds).most_common(1)[0][0])

# Crear el archivo de envío
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': final_predictions})
submission.to_csv('voting_promedio_cada_modelo_2.csv', index=False)

## Ensamble 13: Sigo agregando modelos

In [ ]:
# Cargar las predicciones de los tres modelos
bayes_naive = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulliv3.5.csv')['sentimiento']
random_forest = pd.read_csv('/content/drive/MyDrive/random_forest_smote_modified_2.0.csv')['sentimiento']
xg_boost = pd.read_csv('/content/drive/MyDrive/XGBoost_BagOfWords_trigams_v2.csv')['sentimiento']
red_neuronal = pd.read_csv('/content/drive/MyDrive/red_neuronal_keras_tensor_flow_basico.csv')['sentimiento']
ensamble_voting = pd.read_csv('/content/drive/MyDrive/Voting_Predictions_3.csv')['sentimiento']
bayes_naive_2 = pd.read_csv('/content/drive/MyDrive/bayes_naive_vector_tomando_varias_palabras_3.csv')['sentimiento']
bayes_naive_3 = pd.read_csv('/content/drive/MyDrive/Multinomial_Naive_Bayes_Optimizado.csv')['sentimiento']
bayes_naive_4 = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulli_v3.csv')['sentimiento']
ensamble_voting_2 = pd.read_csv('/content/drive/MyDrive/Voting_Classifier_2.csv')['sentimiento']
red_neuronal_2 = pd.read_csv('/content/drive/MyDrive/keras_tensor_flow_tf_idf_v2.csv')['sentimiento']

# Realizar el ensamble de votación
final_predictions = []
for preds in zip(bayes_naive, random_forest, xg_boost, red_neuronal, ensamble_voting, bayes_naive_2, bayes_naive_3, bayes_naive_4, ensamble_voting_2, red_neuronal_2):
    final_predictions.append(Counter(preds).most_common(1)[0][0])

# Crear el archivo de envío
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': final_predictions})
submission.to_csv('voting_promedio_cada_modelo_3.csv', index=False)

## Ensamble 14: Promedio de los mejores promedios

In [ ]:
promedio_1 = pd.read_csv('/content/drive/MyDrive/voting_promedio_cada_modelo_3.csv')['sentimiento']
promedio_2 = pd.read_csv('/content/drive/MyDrive/voting_promedio_cada_modelo_2.csv')['sentimiento']
promedio_3 = pd.read_csv('/content/drive/MyDrive/voting_promedio_cada_modelo.csv')['sentimiento']

# Realizar el ensamble de votación
final_predictions = []
for preds in zip(promedio_1, promedio_2, promedio_3):
    final_predictions.append(Counter(preds).most_common(1)[0][0])

# Crear el archivo de envío
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': final_predictions})
submission.to_csv('promedio_de_promedios.csv', index=False)

## Ensamble 15

In [ ]:
uno = pd.read_csv('/content/drive/MyDrive/logistic_regression.csv')['sentimiento']
dos = pd.read_csv('/content/drive/MyDrive/Bayes_Naive_Bernoulliv3.5.csv')['sentimiento']
tres = pd.read_csv('/content/drive/MyDrive/voting_promedio_cada_modelo_3.csv')['sentimiento']

# Realizar el ensamble de votación
final_predictions = []
for preds in zip(uno, dos, tres):
    final_predictions.append(Counter(preds).most_common(1)[0][0])

# Crear el archivo de envío
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': final_predictions})
submission.to_csv('promedios_ensamble_15.csv', index=False)

# Logistic Regression

## Basico

### Algoritmo

#### Paso 1: Preprocesamiento y division

In [ ]:
X, y, tfidf_vectorizer, label_encoder = preprocess_data(train)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 2: entreno el modelo

In [ ]:
# Entrenar el modelo de Regresión Logística
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = logistic_model.predict(X_test)

# Convertir label_encoder.classes_ a una lista de strings
label_names = [str(label) for label in label_encoder.classes_]

#### Paso 3: Evaluo el modelo

In [ ]:
# Evaluar el rendimiento del modelo
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_names))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.87      0.89      4961
           1       0.88      0.91      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

Confusion Matrix:
 [[4329  632]
 [ 465 4574]]
Accuracy Score: 0.8903


### Prediccion para el modelo de test dado

In [ ]:
def preprocess_new_data(test, tfidf_vectorizer):
    X_new = tfidf_vectorizer.transform(test['review_es'])
    return X_new

X_new = preprocess_new_data(test_kaggle, tfidf_vectorizer)

# Hacer predicciones en el nuevo conjunto de datos
new_predictions = logistic_model.predict(X_new)

# Convertir las predicciones a etiquetas originales
new_predictions_labels = label_encoder.inverse_transform(new_predictions)

# Añadir las predicciones al DataFrame original
test_kaggle['sentimiento'] = new_predictions_labels

test_kaggle['sentimiento'] = test_kaggle['sentimiento'].replace({0: 'negativo', 1: 'positivo'})

columnas_mostrar = ["ID", "sentimiento"]
logistic_regression = test_kaggle[columnas_mostrar]

# Guardar el DataFrame con las predicciones
logistic_regression.to_csv('logistic_regression.csv', index=False)

## Mejores parametros

### Algoritmo

#### Paso 1: Preprocesamiento y division

In [ ]:
X, y, tfidf_vectorizer, label_encoder = preprocess_data(train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 2: Busco los mejores parametros

In [ ]:
param_grid = {'C': [0.1, 1.0, 10.0]}
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

#### Paso 3: Entreno el modelo

In [ ]:
# Entrenar el mejor modelo de Regresión Logística
best_model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

In [ ]:
label_names = [str(label) for label in label_encoder.classes_]

#### Paso 4: Evaluo el modelo

In [ ]:
# Evaluar el rendimiento del modelo
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_names))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.88      0.89      4961
           1       0.89      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

Confusion Matrix:
 [[4371  590]
 [ 482 4557]]
Accuracy Score: 0.8928


### Prediccion para el modelo de test dado

In [ ]:
# Preprocesar nuevos datos de prueba
X_new = preprocess_new_data(test_kaggle, tfidf_vectorizer)

# Hacer predicciones en el nuevo conjunto de datos
new_predictions = best_model.predict(X_new)

# Convertir las predicciones a etiquetas originales
new_predictions_labels = label_encoder.inverse_transform(new_predictions)

# Añadir las predicciones al DataFrame original
test_kaggle['sentimiento'] = new_predictions_labels

# Mapear etiquetas numéricas a etiquetas originales
test_kaggle['sentimiento'] = test_kaggle['sentimiento'].replace({0: 'negativo', 1: 'positivo'})

# Seleccionar columnas a mostrar
columnas_mostrar = ["ID", "sentimiento"]
logistic_regression = test_kaggle[columnas_mostrar]

# Guardar el DataFrame con las predicciones
logistic_regression.to_csv('logistic_regression_2.csv', index=False)